In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
csv_dd13 = pd.read_csv("dd13.csv")


## Pré-processamento

In [ ]:
csv_dd13.info()

In [ ]:
csv_dd13["Descrição"].fillna('', inplace=True)

In [ ]:
# Retirando valores NaN
csv_dd13 = csv_dd13.dropna()

In [ ]:
# Plotando incidentes por Nível do Acidente
nivel_count = csv_dd13.groupby('Nível do Acidente').count()
plt.figure(figsize=(15,5))
plt.bar(nivel_count.index.values, nivel_count['Descrição'])
plt.xlabel('Nível do Acidente')
plt.ylabel('Numero de incidentes')
plt.show()

In [ ]:
# Funcao para transformar para dense
from sklearn.base import TransformerMixin
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()


 ### Teste de classificação para preencher o nível do acidente

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, precision_score, classification_report,accuracy_score
from sklearn.compose import ColumnTransformer

X = csv_dd13['Descrição']
y = csv_dd13['Nível do Acidente']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state = 13)

In [ ]:
modelLDA = Pipeline(steps = [('tf', TfidfVectorizer()), 
                              ('to_dense', DenseTransformer()), 
                              ('lda', LinearDiscriminantAnalysis())])

In [ ]:
modelLDA.fit(X_train, y_train)

In [ ]:
score=modelLDA.score(X_train, y_train)
ypred=modelLDA.predict(X_test)
cm = confusion_matrix(y_test, ypred)

print("Score: ",score)
print('Accuracy' + str(accuracy_score(y_test, ypred)))
print(cm)
#print(classification_report(y_test, ypred))